In [99]:
%%bash
python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [101]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [102]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

**Create tokenizers**

In [103]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

**Create Fields**

In [104]:
TRG = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

**Download traina nd test data**

In [105]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

**Build Vocabulary**

In [106]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

**Set device, batch size, and create iterators**

In [107]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [108]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

### **Encoder**

In [109]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.rnn = nn.GRU(embed_dim, enc_hid_dim, bidirectional=True)
        self.fc = nn.Linear(2*enc_hid_dim, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        # src = [src len, batch size]
        # embedding = [src len, batch size, embed dim]
        embedding = self.dropout(self.embedding(src))
        
        # output = [src len, batch size, enc hidden dim * no of directions]
        # hidden = [n layers * no of directions, batch size, enc hidden dim]
        outputs, hidden = self.rnn(embedding)

        # print("Embed shape:", embedding.shape)
        # print("Rnn output shape: ",  outputs.shape)
        # print("Rnn hidden shape: ",  hidden.shape) 

        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...] for 
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN 
        # hidden = [batch size, decoder hidden dim]
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)))

        # print("Fc output shape:", hidden.shape)

        return outputs, hidden




In [110]:
# Testing
enc = Encoder(input_dim=32, 
              embed_dim=256, 
              enc_hid_dim=512, 
              dec_hid_dim=512, 
              dropout=0.1)
enc

Encoder(
  (embedding): Embedding(32, 256)
  (rnn): GRU(256, 512, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [111]:
# Testing shapes
e, h = enc.forward(torch.zeros(12, 32).to(torch.int64)) 
e.size()
# embedding: [sentence_length, batch_size, embed_dim]
# output: [sentence_len, batch_size, 2 * enc_hid_dim]
# hidden: [ batch_size, enc_hidden_dim]

torch.Size([12, 32, 1024])

In [112]:
h[0].size()

torch.Size([512])

In [113]:
# h[-1, :, :].size()

### **Attention Block**

In [114]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim*2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, encoder_outputs, hidden):
        # hidden = [batch size, dec hid dim]
        # encoder_outputs = [src len, batch size, enc hid dim * 2]

        src_len = encoder_outputs.shape[0]
        # print("In attention model ---")
        # print("encoder_outputs: ", encoder_outputs.shape)
        # print("hidden: ", hidden.shape)
        

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # hidden = [batch size, src len, dec hid dim]
        # encoder_outputs = [batch size, src len, enc hid dim * 2]
        # [batch, src len, 512] + [batch, src len, 1024] ==> concat along dim 2
        
        # energy is similarity between hidden vectors and encoded vectors
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        # energy = [batch size, src len, dec_hid_dim]

        attention = self.v(energy).squeeze(2) # squeeze unwanted dims
        # attention = [batch size, src len]

        return F.softmax(attention, dim=1) 


In [115]:
# enc = Encoder(input_dim=32, 
#               embed_dim=256, 
#               enc_hid_dim=512, 
#               dec_hid_dim=512, 
#               dropout=0.1)


In [116]:
outputs, hidden = enc.forward(torch.zeros(12, 32).to(torch.int64)) 
outputs.size(), hidden.size()

(torch.Size([12, 32, 1024]), torch.Size([32, 512]))

In [117]:
# only for testing, assume decoder st is same as enc hidden (same dims)
s_hidden = hidden 
s_hidden.size()

torch.Size([32, 512])

In [118]:
# We need this to be added to outputs, which are of size [12, 32, 1024].. restructuring..

tt = s_hidden.unsqueeze(1)
print(tt.size())

rr = tt.repeat(1, 12, 1)
print(rr.size())

outputs.permute(1, 0, 2).size()

torch.Size([32, 1, 512])
torch.Size([32, 12, 512])


torch.Size([32, 12, 1024])

### **Decoder**

In [119]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.attention = attention
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim) # encoder is bidirectional
        self.fc_out = nn.Linear( (enc_hid_dim * 2) + emb_dim + dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        # this hidden is from the decoder hidden output 
        # input is coming as single word, make it as a batch
        # input = [batch size]
        input = input.unsqueeze(0)
        # input = [1, batch size]

        # print("In decoder ---")
        # print("Input shape:", input.shape)
        # print("hidden shape:", hidden.shape)
        # print("Encoder outputs:", encoder_outputs.shape)

        embedding = self.dropout(self.embedding(input))
        # embedding = [1, batch size, emb dim]
        
        attn = self.attention(encoder_outputs, hidden) #        
        # print("Attn shape: ", attn.shape)

        attn = attn.unsqueeze(1)
        # attn = [batch size, 1, src len] # 1 is output from attn

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # encoder_outputs = [batch size, src len, enc hidden dim * 2]

        # bmm is matrix mult
        weighted = torch.bmm(attn, encoder_outputs)
        # weighted = [batch sizze, 1, enc hidden dim*2]

        # print("embedding: ", embedding.shape)
        # print("weighted: ", weighted.shape)

        weighted = weighted.permute(1, 0, 2)
        rnn_input = torch.cat((embedding, weighted), dim=2)
        # rnn_input = [1, batch size, end hidden dim * 2 + emb dim]

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

        # Need to squeeze tensors back, so that extra dim is removed before sending to fc
        embedding = embedding.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedding), dim=1))

        return prediction, hidden.squeeze(0)


### **Seq2Seq Model**

In [120]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

### **Define Parameters**

In [121]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

In [122]:
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

**Init weights**

In [123]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0,std=0.01)
        else:
            nn.init.constant_(param.data, 0)
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7873, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5923, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5923, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [124]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,584,995 trainable parameters


**Define optimizer, loss**

In [125]:
optimizer = optim.Adam(model.parameters())

In [126]:
# ignore padding tokens
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### **Train Loop**

In [127]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

### **Test Loop**

In [128]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [129]:
# Timing function
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### **Train the model**

In [131]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 22s
	Train Loss: 4.756 | Train PPL: 116.331
	 Val. Loss: 4.808 |  Val. PPL: 122.480
Epoch: 02 | Time: 1m 23s
	Train Loss: 3.996 | Train PPL:  54.403
	 Val. Loss: 4.153 |  Val. PPL:  63.637
Epoch: 03 | Time: 1m 23s
	Train Loss: 3.311 | Train PPL:  27.425
	 Val. Loss: 3.599 |  Val. PPL:  36.560
Epoch: 04 | Time: 1m 23s
	Train Loss: 2.788 | Train PPL:  16.252
	 Val. Loss: 3.282 |  Val. PPL:  26.620
Epoch: 05 | Time: 1m 24s
	Train Loss: 2.419 | Train PPL:  11.231
	 Val. Loss: 3.192 |  Val. PPL:  24.346
Epoch: 06 | Time: 1m 23s
	Train Loss: 2.123 | Train PPL:   8.360
	 Val. Loss: 3.138 |  Val. PPL:  23.049
Epoch: 07 | Time: 1m 23s
	Train Loss: 1.891 | Train PPL:   6.627
	 Val. Loss: 3.116 |  Val. PPL:  22.549
Epoch: 08 | Time: 1m 24s
	Train Loss: 1.698 | Train PPL:   5.464
	 Val. Loss: 3.160 |  Val. PPL:  23.575
Epoch: 09 | Time: 1m 23s
	Train Loss: 1.542 | Train PPL:   4.672
	 Val. Loss: 3.213 |  Val. PPL:  24.866
Epoch: 10 | Time: 1m 22s
	Train Loss: 1.420 | Train PPL

In [132]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.215 | Test PPL:  24.892 |
